In [1]:
import xarray as xr
import cf_xarray as cfxr
import isodate

In [2]:
t_analysis = "2025-03-02T0000Z"
fp = f"/home/ec2-user/nwp-forecast-zarr-creator/refs/CONTROL__dmi/{t_analysis}.jsons/isobaricInhPa.json"
ds = xr.open_zarr(f"reference::{fp}")
ds

<xarray.Dataset> Size: 26GB
Dimensions:  (y: 1606, x: 1906, level: 15, time: 10)
Coordinates:
    lat      (y, x) float64 24MB dask.array<chunksize=(1606, 1906), meta=np.ndarray>
  * level    (level) int64 120B 100 150 200 250 300 400 ... 850 900 925 950 1000
    lon      (y, x) float64 24MB dask.array<chunksize=(1606, 1906), meta=np.ndarray>
  * time     (time) datetime64[ns] 80B 2025-03-02 ... 2025-03-02T09:00:00
  * x        (x) float64 15kB -1.528e+06 -1.526e+06 ... 2.28e+06 2.282e+06
  * y        (y) float64 13kB -1.589e+06 -1.587e+06 ... 1.619e+06 1.621e+06
Data variables:
    papt     (time, level, y, x) float64 4GB dask.array<chunksize=(1, 1, 1606, 1906), meta=np.ndarray>
    r        (time, level, y, x) float64 4GB dask.array<chunksize=(1, 1, 1606, 1906), meta=np.ndarray>
    t        (time, level, y, x) float64 4GB dask.array<chunksize=(1, 1, 1606, 1906), meta=np.ndarray>
    u        (time, level, y, x) float64 4GB dask.array<chunksize=(1, 1, 1606, 1906), meta=np.ndarray>
    v        (time, level, y, x) float64 4GB dask.array<chunksize=(1, 1, 1606, 1906), meta=np.ndarray>
    wz       (time, level, y, x) float64 4GB dask.array<chunksize=(1, 1, 1606, 1906), meta=np.ndarray>
    z        (time, level, y, x) float64 4GB dask.array<chunksize=(1, 1, 1606, 1906), meta=np.ndarray>
Attributes:
    edition:            2
    centre:             ekmi
    centreDescription:  Copenhagen
    subCentre:          255
    history:            🪄🧙‍♂️🔮 magic dataset assembly provided by gribscan.Ha...

In [3]:
# copy over cf standard-names where eccodes provides them

for var_name in ds.data_vars:
    if "cfName" in ds[var_name].attrs:
        ds[var_name].attrs["standard_name"] = ds[var_name].attrs["cfName"]

In [4]:
ds.cf

Coordinates:
             CF Axes:   X, Y, Z, T: n/a

      CF Coordinates:   longitude: ['lon']
                        latitude: ['lat']
                        vertical, time: n/a

       Cell Measures:   area, volume: n/a

      Standard Names:   latitude: ['lat']
                        longitude: ['lon']

              Bounds:   n/a

       Grid Mappings:   n/a

Data Variables:
       Cell Measures:   area, volume: n/a

      Standard Names:   air_temperature: ['t']
                        eastward_wind: ['u']
                        geopotential: ['z']
                        northward_wind: ['v']
                        relative_humidity: ['r']

              Bounds:   n/a

       Grid Mappings:   n/a

In [5]:
DANRA_CRS_WKT = """
PROJCRS["DMI HARMONIE DANRA lambert projection",
    BASEGEOGCRS["DMI HARMONIE DANRA lambert CRS",
        DATUM["DMI HARMONIE DANRA lambert datum",
            ELLIPSOID["Sphere", 6367470, 0,
                LENGTHUNIT["metre", 1, ID["EPSG", 9001]]
            ]
        ],
        PRIMEM["Greenwich", 0,
            ANGLEUNIT["degree", 0.0174532925199433, ID["EPSG", 8901]]
        ],
        ID["EPSG",4035]
    ],
    CONVERSION["Lambert Conic Conformal (2SP)",
        METHOD["Lambert Conic Conformal (2SP)", ID["EPSG", 9802]],
        PARAMETER["Latitude of false origin", 56.7,
            ANGLEUNIT["degree", 0.0174532925199433, ID["EPSG", 8821]]
        ],
        PARAMETER["Longitude of false origin", 25,
            ANGLEUNIT["degree", 0.0174532925199433, ID["EPSG", 8822]]
        ],
        PARAMETER["Latitude of 1st standard parallel", 56.7,
            ANGLEUNIT["degree", 0.0174532925199433, ID["EPSG", 8823]]
        ],
        PARAMETER["Latitude of 2nd standard parallel", 56.7,
            ANGLEUNIT["degree", 0.0174532925199433, ID["EPSG", 8824]]
        ],
        PARAMETER["Easting at false origin", 0,
            LENGTHUNIT["metre", 1, ID["EPSG", 8826]]
        ],
        PARAMETER["Northing at false origin", 0,
            LENGTHUNIT["metre", 1, ID["EPSG", 8827]]
        ]
    ],
    CS[Cartesian, 2],
    AXIS["(E)", east, ORDER[1],
        LENGTHUNIT["metre", 1, ID["EPSG", 9001]]
    ],
    AXIS["(N)", north, ORDER[2],
        LENGTHUNIT["metre", 1, ID["EPSG", 9001]]
    ],
    USAGE[
        AREA["Denmark and surrounding regions"],
        BBOX[47, -3, 65, 25],
        SCOPE["Danra reanalysis projection"]
    ]
]
"""


DANRA_CRS_ATTRS = {
    "crs_wkt": "".join(DANRA_CRS_WKT.splitlines()),
    "semi_major_axis": 6367470.0,
    "semi_minor_axis": 6367470.0,
    "inverse_flattening": 0.0,
    "reference_ellipsoid_name": "Sphere",
    "longitude_of_prime_meridian": 0.0,
    "prime_meridian_name": "Greenwich",
    "geographic_crs_name": "DMI HARMONIE DANRA lambert CRS",
    "horizontal_datum_name": "DMI HARMONIE DANRA lambert datum",
    "projected_crs_name": "DMI HARMONIE DANRA lambert projection",
    "grid_mapping_name": "lambert_conformal_conic",
    "standard_parallel": [0.0567, 0.0567],
    "latitude_of_projection_origin": 0.0567,
    "longitude_of_central_meridian": 0.025,
    "false_easting": 0.0,
    "false_northing": 0.0,
}


def _add_projection_info_to_all_variables(ds):
    """
    Add CF-compliant (http://cfconventions.org/cf-conventions/cf-conventions.html#appendix-grid-mappings)
    projection info by adding a new variable that holds the projection attributes and setting on each variable
    that this projection applies.

    NOTE: currently gribscan doesn't return the projection information attributes when parsing
    Harmonie GRIB files. For that reason the projection parameters are hardcoded here. They
    should be moved in future so that this information is returned by gribscan and set in dmidc

    Parameters
    ----------
    ds: xr.Dataset
        the dataset to add projection info to

    Returns
    -------
    Nothing
    """
    PROJECTION_IDENTIFIER = "danra_projection"
    ds[PROJECTION_IDENTIFIER] = xr.DataArray()
    ds[PROJECTION_IDENTIFIER].attrs.update(DANRA_CRS_ATTRS)

    for var_name in ds.data_vars:
        ds[var_name].attrs["grid_mapping"] = PROJECTION_IDENTIFIER

In [6]:
_add_projection_info_to_all_variables(ds)

In [7]:
ds.cf

Coordinates:
             CF Axes:   X, Y, Z, T: n/a

      CF Coordinates:   longitude: ['lon']
                        latitude: ['lat']
                        vertical, time: n/a

       Cell Measures:   area, volume: n/a

      Standard Names:   latitude: ['lat']
                        longitude: ['lon']

              Bounds:   n/a

       Grid Mappings:   n/a

Data Variables:
       Cell Measures:   area, volume: n/a

      Standard Names:   air_temperature: ['t']
                        eastward_wind: ['u']
                        geopotential: ['z']
                        northward_wind: ['v']
                        relative_humidity: ['r']

              Bounds:   n/a

       Grid Mappings:   lambert_conformal_conic: ['danra_projection']

In [8]:
import sys
sys.path.insert(0, "../")
from zarr_creator.write_zarr import write_zarr_to_s3

In [9]:
rechunk_to = dict(time=1, x=ds.x.size // 2, y=ds.y.size // 2)
# check that with the chunking provided that the arrays exactly fit into the chunks
for dim in rechunk_to:
    assert ds[dim].size % rechunk_to[dim] == 0

In [10]:
write_zarr_to_s3(ds=ds,
    dataset_id="pressure_levels", 
    rechunk_to=rechunk_to, member="control",
    t_analysis=isodate.parse_datetime(t_analysis)
)

UnboundLocalError: local variable 'temp_dir' referenced before assignment

: 

: 

: 

: 